In [1]:
%matplotlib inline
import os
import calendar
from pathlib import Path
import numpy as np
import networkx as nx
import pandas as pd
from pandas.plotting import scatter_matrix, parallel_coordinates
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pylab as plt

In [63]:
#read the dataset
movies = pd.read_csv('C:/Users/Win Nguyen/Desktop/MSA 550/Gender-Hollywood/US.movies.2.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5243 entries, 0 to 5242
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     5243 non-null   object 
 1   rating    5235 non-null   object 
 2   genre     5243 non-null   object 
 3   year      5243 non-null   int64  
 4   score     5243 non-null   float64
 5   votes     5243 non-null   int64  
 6   director  5243 non-null   object 
 7   d_gender  5243 non-null   int64  
 8   writer    5243 non-null   object 
 9   w_gender  5243 non-null   int64  
 10  star      5243 non-null   object 
 11  s_gender  5243 non-null   int64  
 12  country   5243 non-null   object 
 13  budget    4878 non-null   float64
 14  gross     5220 non-null   float64
 15  company   5239 non-null   object 
 16  runtime   5242 non-null   float64
dtypes: float64(4), int64(5), object(8)
memory usage: 696.5+ KB


In [64]:
#summary statistic
movies.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,year,score,votes,d_gender,w_gender,s_gender,budget,gross,runtime
count,5243.00000,5243.00000,5243.00000,5243.00000,5243.00000,5243.00000,4878.00000,5220.00000,5242.00000
mean,2000.22869,6.28640,99112.54358,0.06676,0.11387,0.24146,34025292.04674,92518565.86782,106.18295
std,10.84759,0.93914,172864.21393,0.24962,0.31768,0.42801,41561382.93594,178867095.00742,16.88642
min,1980.00000,1.90000,35.00000,0.00000,0.00000,0.00000,4000.00000,309.00000,63.00000
25%,1991.00000,5.70000,11000.00000,0.00000,0.00000,0.00000,8000000.00000,7785895.25000,95.00000
50%,2000.00000,6.40000,39000.00000,0.00000,0.00000,0.00000,20000000.00000,28594520.50000,103.00000
75%,2010.00000,6.90000,108000.00000,0.00000,0.00000,0.00000,40000000.00000,97114969.00000,115.00000
max,2020.00000,9.30000,2400000.00000,1.00000,1.00000,1.00000,356000000.00000,2847246203.00000,271.00000


In [65]:
# see all missing values
movies[movies.isna().any(axis=1)]



,title,rating,genre,year,score,votes,director,d_gender,writer,w_gender,star,s_gender,country,budget,gross,company,runtime
1001,Heart of Dixie,PG,Drama,1989,5.2,677,Martin Davidson,0,Anne Rivers Siddons,1,Ally Sheedy,0,United States,8000000.0,1097333.0,NaN,95.0
1002,Tap,PG-13,Comedy,1989,6.5,1500,Nick Castle,0,Nick Castle,0,Gregory Hines,0,United States,NaN,9114702.0,Beco Films,111.0
1004,The Phantom of the Opera,R,Drama,1989,5.5,3800,Dwight H. Little,0,Gaston Leroux,0,Robert Englund,0,United States,NaN,3953745.0,21st Century Film Corporation,93.0
1005,Triumph of the Spirit,R,Biography,1989,6.8,2300,Robert M. Young,0,Shimon Arama,0,Willem Dafoe,0,United States,NaN,408839.0,Nova International Films,120.0
1006,The Unbelievable Truth,R,Comedy,1989,7.3,5000,Hal Hartley,0,Hal Hartley,0,Adrienne Shelly,0,United States,75000.0,NaN,Action Features,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004,Wonderstruck,PG,Adventure,2017,6.2,9800,Todd Haynes,0,Brian Selznick,0,Oakes Fegley,0,United States,NaN,3285916.0,Amazon Studios,116.0
5007,The Wall,R,Action,2017,6.2,25000,Doug Liman,0,Dwain Worrell,0,Aaron Taylor-Johnson,0,United States,NaN,4477141.0,Amazon Studios,88.0
5222,Ms. Purple,NaN,Crime,2019,6.2,481,Justin Chon,0,Justin Chon,0,Jake Choi,0,United States,45000.0,80657.0,Electric Panda Entertainment,87.0
5230,Give Me Liberty,NaN,Comedy,2019,6.7,1700,Kirill Mikhanovsky,0,Alice Austen,1,Chris Galust,0,United States,1000000.0,624818.0,Give Me Liberty,110.0


In [66]:
#check nulls value with condition d_gender = 1
d_femme = movies.loc[movies.d_gender.isin([1])]
d_femme.describe(include = "all")
d_femme[d_femme.budget.isna()]
d_femme.describe().apply(lambda s: s.apply('{0:.5f}'.format))


,year,score,votes,d_gender,w_gender,s_gender,budget,gross,runtime
count,350.00000,350.00000,350.00000,350.00000,350.00000,350.00000,350.00000,348.00000,349.00000
mean,2003.07714,6.19857,58138.46571,1.00000,0.58286,0.59714,20743105.06857,58122630.31034,103.94842
std,10.71658,0.83890,121830.41881,0.00000,0.49379,0.49117,29786523.78443,125910518.83359,13.88573
min,1980.00000,3.70000,406.00000,1.00000,0.00000,0.00000,4000.00000,5073.00000,76.00000
25%,1995.00000,5.70000,8800.00000,1.00000,0.00000,0.00000,2625000.00000,2475449.75000,93.00000
50%,2004.00000,6.30000,22000.00000,1.00000,1.00000,1.00000,11000000.00000,15402347.50000,102.00000
75%,2012.00000,6.80000,54750.00000,1.00000,1.00000,1.00000,25000000.00000,50026563.50000,111.00000
max,2020.00000,8.70000,1700000.00000,1.00000,1.00000,1.00000,200000000.00000,1128462972.00000,157.00000


In [67]:
#check nulls value with condition d_gender = 0
d_mas = movies.loc[movies.d_gender.isin([0])]
d_mas.describe(include = "all")
d_mas[d_mas.budget.isna()]
d_mas.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,year,score,votes,d_gender,w_gender,s_gender,budget,gross,runtime
count,4893.00000,4893.00000,4893.00000,4893.00000,4893.00000,4893.00000,4528.00000,4872.00000,4893.00000
mean,2000.02493,6.29268,102043.45044,0.00000,0.08032,0.21602,35051962.85998,94975418.40764,106.34233
std,10.82930,0.94567,175591.46150,0.00000,0.27181,0.41157,42164002.46070,181821957.41780,17.07034
min,1980.00000,1.90000,35.00000,0.00000,0.00000,0.00000,4000.00000,309.00000,63.00000
25%,1991.00000,5.70000,12000.00000,0.00000,0.00000,0.00000,9000000.00000,8302899.25000,95.00000
50%,2000.00000,6.40000,41000.00000,0.00000,0.00000,0.00000,20000000.00000,29966845.00000,103.00000
75%,2009.00000,6.90000,112000.00000,0.00000,0.00000,0.00000,44000000.00000,100766243.00000,115.00000
max,2020.00000,9.30000,2400000.00000,0.00000,1.00000,1.00000,356000000.00000,2847246203.00000,271.00000


In [68]:
#find the mean of movies with score >= 6 and less than 7 that was made by male directors
#substistute the missing values of movies' budget that made by male directors, and has score higher than 6 but less than 7
#with the mean 
movies.loc[(movies.d_gender.isin([0]))  & (movies.score>=6) & (movies.score <7)] = movies.loc[(movies.d_gender.isin([0]))  & (movies.score>=6) & (movies.score <7)].fillna(movies.loc[(movies.d_gender.isin([0]))  & (movies.score>=6) & (movies.score <7)].mean())
movies[movies.budget.isna()]

,title,rating,genre,year,score,votes,director,d_gender,writer,w_gender,star,s_gender,country,budget,gross,company,runtime
1004,The Phantom of the Opera,R,Drama,1989,5.5,3800,Dwight H. Little,0,Gaston Leroux,0,Robert Englund,0,United States,NaN,3953745.0,21st Century Film Corporation,93.0
1015,Listen to Me,PG-13,Drama,1989,5.3,1000,Douglas Day Stewart,0,Douglas Day Stewart,0,Kirk Cameron,0,United States,NaN,4299023.0,Weintraub Entertainment Group,107.0
1017,Wired,R,Biography,1989,3.7,790,Larry Peerce,0,Bob Woodward,0,Michael Chiklis,0,United States,NaN,1089000.0,FM Entertainment,112.0
1019,Rooftops,R,Action,1989,4.8,490,Robert Wise,0,Allan A. Goldstein,0,Jason Gedrick,0,United States,NaN,2043889.0,Koch Company,98.0
1025,The Terror Within,R,Horror,1989,4.8,1600,Thierry Notz,0,Thomas McKelvey Cleaver,0,Andrew Stevens,0,United States,NaN,858591.0,Concorde Pictures,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978,Columbus,Not Rated,Drama,2017,7.2,15000,Kogonada,0,Kogonada,0,John Cho,0,United States,NaN,1093077.0,Depth of Field,100.0
4986,First Reformed,R,Drama,2017,7.1,51000,Paul Schrader,0,Paul Schrader,0,Ethan Hawke,0,United States,NaN,3862498.0,Killer Films,113.0
4991,Song to Song,R,Drama,2017,5.7,20000,Terrence Malick,0,Terrence Malick,0,Ryan Gosling,0,United States,NaN,1813453.0,Buckeye Pictures,129.0
4998,Jeepers Creepers III,Not Rated,Horror,2017,3.9,18000,Victor Salva,0,Victor Salva,0,Stan Shaw,0,United States,NaN,4017852.0,Infinity Films,100.0


In [69]:
#find the mean of movies with score less than 6 that was made by male directors
#substistute the missing values of movies' budget made by male director and has score less than 6
#with the correspoding mean 
movies.loc[(movies.d_gender.isin([0]))  & (movies.score<6)] = movies.loc[(movies.d_gender.isin([0]))  & (movies.score<6)].fillna(movies.loc[(movies.d_gender.isin([0]))  & (movies.score<6)].mean())
movies[movies.budget.isna()]

,title,rating,genre,year,score,votes,director,d_gender,writer,w_gender,star,s_gender,country,budget,gross,company,runtime
1051,Romero,PG-13,Biography,1989,7.2,2100,John Duigan,0,John Sacret Young,0,Raul Julia,0,United States,NaN,1316495.0,Paulist Pictures,102.0
1118,Reversal of Fortune,R,Biography,1990,7.2,15000,Barbet Schroeder,0,Alan Dershowitz,0,Jeremy Irons,0,United States,NaN,15445131.0,Sovereign Pictures,111.0
1189,To Sleep with Anger,PG,Drama,1990,7.2,2200,Charles Burnett,0,Charles Burnett,0,Danny Glover,0,United States,NaN,1161135.0,SVS Films,102.0
1256,Wild Hearts Can't Be Broken,G,Drama,1991,7.2,5000,Steve Miner,0,Matt Williams,0,Gabrielle Anwar,0,United States,NaN,7294835.0,Walt Disney Pictures,88.0
1261,Defending Your Life,PG,Comedy,1991,7.2,18000,Albert Brooks,0,Albert Brooks,0,Albert Brooks,0,United States,NaN,16371128.0,Geffen Pictures,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753,The End of the Tour,R,Biography,2015,7.3,29000,James Ponsoldt,0,Donald Margulies,0,Jason Segel,0,United States,NaN,3072991.0,Modern Man Films,106.0
4950,Papillon,R,Adventure,2017,7.2,63000,Michael Noer,0,Aaron Guzikowski,0,Charlie Hunnam,0,United States,NaN,10060903.0,Czech Anglo Productions,133.0
4959,Lucky,Not Rated,Comedy,2017,7.4,21000,John Carroll Lynch,0,Logan Sparks,0,Harry Dean Stanton,0,United States,NaN,2728446.0,Superlative Films,88.0
4978,Columbus,Not Rated,Drama,2017,7.2,15000,Kogonada,0,Kogonada,0,John Cho,0,United States,NaN,1093077.0,Depth of Field,100.0


In [70]:
#find the mean of movies with score more than 7 that was made by male directors
#substistute the missing values of movies' budget made by male director and has score more than 7
#with the correspoding mean 
movies.loc[(movies.d_gender.isin([0]))  & (movies.score>=7)] = movies.loc[(movies.d_gender.isin([0]))  & (movies.score>=7)].fillna(movies.loc[(movies.d_gender.isin([0]))  & (movies.score>=7)].mean())
movies[movies.budget.isna()]



,title,rating,genre,year,score,votes,director,d_gender,writer,w_gender,star,s_gender,country,budget,gross,company,runtime


In [71]:
#Female screenplay writer . check nulls value with condition w_gender = 1
w_femme = movies.loc[movies.w_gender.isin([1])]
w_femme.describe(include = "all")
w_femme[w_femme.budget.isna()]
w_femme.describe().apply(lambda s: s.apply('{0:.5f}'.format))


,year,score,votes,d_gender,w_gender,s_gender,budget,gross,runtime
count,597.00000,597.00000,597.00000,597.00000,597.00000,597.00000,597.00000,595.00000,596.00000
mean,2002.19933,6.20251,72339.62479,0.34171,1.00000,0.64322,25879628.24645,80347534.04202,105.41107
std,10.80713,0.88244,147991.71895,0.47468,0.00000,0.47945,34522826.08208,159496293.80897,14.08893
min,1980.00000,2.30000,253.00000,0.00000,1.00000,0.00000,4000.00000,5073.00000,76.00000
25%,1994.00000,5.70000,9400.00000,0.00000,1.00000,0.00000,5000000.00000,5985597.50000,96.00000
50%,2003.00000,6.30000,27000.00000,0.00000,1.00000,1.00000,16000000.00000,24959607.00000,103.00000
75%,2012.00000,6.80000,71000.00000,1.00000,1.00000,1.00000,30000000.00000,81331979.50000,115.00000
max,2020.00000,8.70000,1700000.00000,1.00000,1.00000,1.00000,250000000.00000,1450026933.00000,172.00000


In [72]:
#check nulls value with condition s_gender = 1
s_femme = movies.loc[movies.s_gender.isin([1])]
s_femme.describe(include = "all")
s_femme[s_femme.budget.isna()]
s_femme.describe().apply(lambda s: s.apply('{0:.5f}'.format))



,year,score,votes,d_gender,w_gender,s_gender,budget,gross,runtime
count,1266.00000,1266.00000,1266.00000,1266.00000,1266.00000,1266.00000,1266.00000,1264.00000,1265.00000
mean,2002.12954,6.11722,69487.83965,0.16509,0.30332,1.00000,25729065.91975,78102943.91930,103.95178
std,11.06848,0.89905,112664.13763,0.37141,0.45987,0.00000,35034082.39502,160542748.38666,14.11391
min,1980.00000,2.10000,216.00000,0.00000,0.00000,1.00000,4000.00000,3661.00000,76.00000
25%,1993.00000,5.60000,9925.00000,0.00000,0.00000,1.00000,5000000.00000,5984862.25000,94.00000
50%,2003.00000,6.20000,30000.00000,0.00000,0.00000,1.00000,15000000.00000,25529298.00000,102.00000
75%,2012.00000,6.70000,80000.00000,0.00000,1.00000,1.00000,30000000.00000,83107708.75000,111.00000
max,2020.00000,8.60000,1300000.00000,1.00000,1.00000,1.00000,317000000.00000,2069521700.00000,187.00000


# Bechdel Test Scracping

In [15]:
import pandas as pd

from bs4 import BeautifulSoup
from random import randint
from requests import get
from time import sleep, time
from warnings import warn

In [16]:
help(BeautifulSoup)

Help on class BeautifulSoup in module bs4:

class BeautifulSoup(bs4.element.Tag)
 |  BeautifulSoup(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)
 |  
 |  A data structure representing a parsed HTML or XML document.
 |  
 |  Most of the methods you'll call on a BeautifulSoup object are inherited from
 |  PageElement or Tag.
 |  
 |  Internally, this class defines the basic interface called by the
 |  tree builders when converting an HTML/XML document into a data
 |  structure. The interface abstracts away the differences between
 |  parsers. To write a new tree builder, you'll need to understand
 |  these methods as a whole.
 |  
 |  These methods will be called by the BeautifulSoup constructor:
 |    * reset()
 |    * feed(markup)
 |  
 |  The tree builder may call these methods from its feed() implementation:
 |    * handle_starttag(name, attrs) # See note about return value
 |    * handle_endtag(na

In [17]:
import urllib.request as url
base_url = 'https://bechdeltest.com/'
# open connection, grab page
client = url.urlopen(base_url)

# store html in variable
base_html = client.read()





In [18]:

# parse html
soup = BeautifulSoup(base_html, 'html.parser')

In [19]:
soup.h1

<h1 style="margin-bottom: 5px"><a href="/" style="text-decoration:none;">Bechdel Test Movie List</a></h1>

In [20]:
soup.p

<p>The <b>Bechdel Test</b>, or <b><a href="https://www.independent.co.uk/arts-entertainment/films/news/please-stop-calling-it-the-bechdel-test-says-alison-bechdel-10474730.html">Bechdel-Wallace Test</a></b>, sometimes called the <i>Mo Movie Measure</i> or <i>Bechdel Rule</i> is a simple test which names the following three criteria: (1) it has to have at least two women in it, who (2) who talk to each other, about (3) something besides a man. The test was popularized by <b>Alison Bechdel</b>'s comic, the name of which Google won't let me put on this page for inciting hate, in a 1985 strip called <a href="https://dykestowatchoutfor.com/the-rule/"><i>The Rule</i></a>. For a nice video introduction to the subject please check out <a href="http://www.feministfrequency.com/2009/12/the-bechdel-test-for-women-in-movies/">The Bechdel Test for Women in Movies</a> on <a href="http://www.feministfrequency.com/">feministfrequency.com</a>.</p>

In [21]:
# grab each movie
movies = soup.findAll('div', {'class': 'movie'})


200

In [22]:
len(movies)

200

In [23]:
movies[0]

<div class="movie"><a href="http://us.imdb.com/title/tt14843560/"><img alt="[[0]]" src="/static/nopass.png" title="[Fewer than two women in this movie]"/></a>
<a href="/view/10065/#nofilter/" id="movie-10065">#Nofilter</a> <a href="/view/10065/#nofilter/" onclick="showComments('10065'); return false;"><img alt="[1 comment(s) available]" id="comment-img-10065" src="/static/comments.png" style="height: 10px; width: 10px;" title="1 comment"/></a> </div>

In [26]:
# movies object contains imdb id (within imdb link), 
#Bechdel Test score, 'pass' or 'nopass', movie title in 3 
# different formats, and number of comments
movie = movies[0]


In [27]:
len(movie.contents)

6

In [28]:
movie.contents

[<a href="http://us.imdb.com/title/tt14843560/"><img alt="[[0]]" src="/static/nopass.png" title="[Fewer than two women in this movie]"/></a>,
 '\n',
 <a href="/view/10065/#nofilter/" id="movie-10065">#Nofilter</a>,
 ' ',
 <a href="/view/10065/#nofilter/" onclick="showComments('10065'); return false;"><img alt="[1 comment(s) available]" id="comment-img-10065" src="/static/comments.png" style="height: 10px; width: 10px;" title="1 comment"/></a>,
 ' ']

In [31]:
movie.parent.h3.a['id'].split('-')[1]

'2021'

In [32]:
movies[1].contents[2].text

'Afterlife of the Party'

In [33]:
movie.a

<a href="http://us.imdb.com/title/tt14843560/"><img alt="[[0]]" src="/static/nopass.png" title="[Fewer than two women in this movie]"/></a>

In [34]:
movie.a.img['alt']

'[[0]]'

In [35]:

#does not pass the test
movie.a.img['alt'][2]

'0'

In [36]:

movie.a['href']

'http://us.imdb.com/title/tt14843560/'

In [37]:

movie.a['href'].split('/')

['http:', '', 'us.imdb.com', 'title', 'tt14843560', '']

In [38]:
movie.a['href'].split('/')[4]

'tt14843560'

In [39]:
movie.a.img['src'].split('/')[2].split('.')[0]

'nopass'

In [43]:
# test run on page one (sorting by year)

years = []
movie_titles = []
test_scores = []
pass_or_no_pass = []


for movie in movies:
    
    year = movie.find_previous('h3').a['id'].split('-')[1]
    years.append(int(year))
    
    title = movie.contents[2].text.strip()
    movie_titles.append(title)
    
    test_score = movie.a.img['alt'][2]
    test_scores.append(int(test_score))
    
    passing = movie.a.img['src'].split('/')[2].split('.')[0]
    pass_or_no_pass.append(passing)
    
   

In [44]:
test_df = pd.DataFrame({'year': years,
                       'title': movie_titles,
                       'score': test_scores,
                       'passing': pass_or_no_pass})

In [45]:

test_df.head()

,year,title,score,passing
0,2021,#Nofilter,0,nopass
1,2021,Afterlife of the Party,3,pass
2,2021,Ajeeb Daastaans,3,pass
3,2021,America: The Motion Picture,3,pass
4,2021,Army of the Dead,3,pass


In [46]:
from time import sleep, time
from random import randint
from requests import get
from warnings import warn

In [ ]:
# page format - first page was page 0 I guess
# https://bechdeltest.com/?page=1
# last page is page 40
# use clear_output(wait=True) at end of loop if scraping something really big
# from Ipython.core.display import clear_output

In [47]:
# pages = [str(i) for i in range(1,41)]

start = time()
requests = 0

# reset lists
years = []
movie_titles = []
test_scores = []
pass_or_no_pass = []

# for r in range(41):
for r in range(5):
    # request
    
    requests += 1
    sleep(randint(1,5))
    elapsed = time() - start
    
    print(f'Request: {requests} Frequency: {round(requests/elapsed, 3)} requests/second')

Request: 1 Frequency: 0.999 requests/second
Request: 2 Frequency: 0.995 requests/second
Request: 3 Frequency: 0.498 requests/second
Request: 4 Frequency: 0.363 requests/second
Request: 5 Frequency: 0.415 requests/second


In [48]:
# reset lists

years = []
movie_titles = []
test_scores = []
pass_or_no_pass = []

start = time()
requests = 0

# first page will execute outside of loop due to different urls

base_url = 'https://bechdeltest.com/'
response = get(base_url)

# I swear I am not a robot beep boop beep
sleep(randint(8,15))

requests += 1
elapsed = time() - start
print(f'Request: {requests} Frequency: {round(requests/elapsed, 3)} requests/second')

# warning for non-200 status codes
if response.status_code != 200:
    warn(f'Request: {requests} Status code: {response.status_code}')
    
soup = BeautifulSoup(response.text, 'html.parser')
movies = soup.findAll('div', {'class': 'movie'})

for movie in movies:
    
    year = movie.find_previous('h3').a['id'].split('-')[1]
    years.append(int(year))
    
    title = movie.contents[2].text.strip()
    movie_titles.append(title)
    
    test_score = movie.a.img['alt'][2]
    test_scores.append(int(test_score))
    
    passing = movie.a.img['src'].split('/')[2].split('.')[0]
    pass_or_no_pass.append(passing)
    

    
# now start with page 1
pages = [str(i) for i in range(1,41)]

for page in pages:
    
    response = get(base_url + '?page=' + page)
    
    # repeat sleeping, print statements, and warnings
    sleep(randint(8,15))

    requests += 1
    elapsed = time() - start
    print(f'Request: {requests} Frequency: {round(requests/elapsed, 3)} requests/second')

    
    if response.status_code != 200:
        warn(f'Request: {requests} Status code: {response.status_code}')
        
    if requests > 42:
        warn('Number of requests greater than expected.')
        break
    
    soup = BeautifulSoup(response.text, 'html.parser')
    movies = soup.findAll('div', {'class': 'movie'})
    
    for movie in movies:
        
        year = movie.find_previous('h3').a['id'].split('-')[1]
        years.append(int(year))
    
        title = movie.contents[2].text.strip()
        movie_titles.append(title)
    
        test_score = movie.a.img['alt'][2]
        test_scores.append(int(test_score))
    
        passing = movie.a.img['src'].split('/')[2].split('.')[0]
        pass_or_no_pass.append(passing)
    
       

Request: 1 Frequency: 0.063 requests/second
Request: 2 Frequency: 0.073 requests/second
Request: 3 Frequency: 0.074 requests/second
Request: 4 Frequency: 0.07 requests/second
Request: 5 Frequency: 0.071 requests/second
Request: 6 Frequency: 0.072 requests/second
Request: 7 Frequency: 0.072 requests/second
Request: 8 Frequency: 0.073 requests/second
Request: 9 Frequency: 0.073 requests/second
Request: 10 Frequency: 0.073 requests/second
Request: 11 Frequency: 0.073 requests/second
Request: 12 Frequency: 0.072 requests/second
Request: 13 Frequency: 0.071 requests/second
Request: 14 Frequency: 0.071 requests/second
Request: 15 Frequency: 0.073 requests/second
Request: 16 Frequency: 0.073 requests/second
Request: 17 Frequency: 0.072 requests/second
Request: 18 Frequency: 0.072 requests/second
Request: 19 Frequency: 0.073 requests/second
Request: 20 Frequency: 0.074 requests/second
Request: 21 Frequency: 0.074 requests/second
Request: 22 Frequency: 0.074 requests/second
Request: 23 Frequenc

In [49]:
bechdel_df = pd.DataFrame({'year': years, 
                           'title': movie_titles,
                           'score': test_scores,
                           'passing': pass_or_no_pass})

In [50]:
bechdel_df.head(15)

,year,title,score,passing
0,2021,#Nofilter,0,nopass
1,2021,Afterlife of the Party,3,pass
2,2021,Ajeeb Daastaans,3,pass
3,2021,America: The Motion Picture,3,pass
4,2021,Army of the Dead,3,pass
5,2021,Barb and Star Go to Vista Del Mar,3,pass
6,2021,Black Widow,3,pass
7,2021,Cherry,0,nopass
8,2021,Cinderella,3,pass
9,2021,CODA,3,pass


In [51]:
bechdel_df.describe(include = 'all')

,year,title,score,passing
count,8200.000000,8200,8200.000000,8200
unique,NaN,8046,NaN,2
top,NaN,Robin Hood,NaN,pass
freq,NaN,4,NaN,4825
mean,2003.537073,NaN,2.180244,NaN
std,13.313472,NaN,1.071741,NaN
min,1964.000000,NaN,0.000000,NaN
25%,1997.000000,NaN,1.000000,NaN
50%,2008.000000,NaN,3.000000,NaN
75%,2013.000000,NaN,3.000000,NaN


In [55]:
#change the passing column into binary
bechdel_df.passing.replace({'nopass': 0, 'pass': 1}, inplace=True)

In [56]:

bechdel_df.head()

,year,title,score,passing
0,2021,#Nofilter,0,0
1,2021,Afterlife of the Party,3,1
2,2021,Ajeeb Daastaans,3,1
3,2021,America: The Motion Picture,3,1
4,2021,Army of the Dead,3,1


In [60]:
#find and drop duplicate
dupes = bechdel_df.duplicated()
for idx, status in enumerate(dupes):
    if status == True:
        print(idx)
        
bechdel_df.drop_duplicates(inplace=True)
bechdel_df.duplicated().sum()

0

In [62]:
bechdel_df.to_csv('C:/Users/Win Nguyen/Desktop/MSA 550/Gender-Hollywood/bechdel_test_movies.csv', index=False)

In [78]:
#attempt to merge my two csv files
bechdel_test_movies = pd.read_csv('C:/Users/Win Nguyen/Desktop/MSA 550/Gender-Hollywood/bechdel_test_movies.csv')

bechdel_test_movies = bechdel_test_movies.dropna(axis=1)
movies_test = movies.merge(bechdel_test_movies, on='title')
movies_test.to_csv('C:/Users/Win Nguyen/Desktop/MSA 550/Gender-Hollywood/movies_test.csv', index=False)

In [79]:
movies_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2961 entries, 0 to 2960
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     2961 non-null   object 
 1   rating    2959 non-null   object 
 2   genre     2961 non-null   object 
 3   year_x    2961 non-null   int64  
 4   score_x   2961 non-null   float64
 5   votes     2961 non-null   int64  
 6   director  2961 non-null   object 
 7   d_gender  2961 non-null   int64  
 8   writer    2961 non-null   object 
 9   w_gender  2961 non-null   int64  
 10  star      2961 non-null   object 
 11  s_gender  2961 non-null   int64  
 12  country   2961 non-null   object 
 13  budget    2961 non-null   float64
 14  gross     2960 non-null   float64
 15  company   2961 non-null   object 
 16  runtime   2960 non-null   float64
 17  year_y    2961 non-null   int64  
 18  score_y   2961 non-null   int64  
 19  passing   2961 non-null   int64  
dtypes: float64(4), int64(8), objec